In [15]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from numpy.linalg import norm
from collections import Counter, defaultdict
from scipy.sparse import csr_matrix

import numpy as np
import scipy as sp
from collections import defaultdict
from collections import Counter
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import norm
import re
import copy
import random
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import metrics
from stemming.porter2 import stem
import numpy as np
import scipy as sp
%matplotlib inline
import matplotlib.pyplot as plt
from collections import defaultdict

import time
import numpy as np

import nltk as nltk

from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from pprint import pprint
import string
import collections
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import urllib2
import numpy as np
import pandas as pd
import scipy.sparse as sp
from numpy.linalg import norm
from collections import Counter, defaultdict
from scipy.sparse import csr_matrix

In [16]:

def filterLen(docs, minlen):
    """ filter out terms that are too short.
    docs is a list of lists, each inner list is a document represented as a
    list of words minlen is the minimum length of the word to keep
    
    """
    with open("stop-words.txt", "r") as fh:
        stop_words = fh.readlines()
        
    st_list=[]
    for s in stop_words:
        s=s.replace("\n", "")
        st_list.append(s)
    
    return [ [t for t in d if (len(t) >= minlen and t not in st_list) ] for d in docs ]

def stemDoc(docs):
    """ automatically removes suffixes (and in some cases prefixes) in order to
    find the root word or stem of a given word
    """
    
    return [ [stem(t) for t in d ] for d in docs]


def getPositiveNegativeWords():
    with open("negative-words.txt", "r") as fh:
        negative_words = fh.readlines()
    with open("positive-words.txt", "r") as fh:
        positive_words = fh.readlines()

    
    pos_list=[]
    for s in positive_words:
        s=s.replace("\n", "")
        pos_list.append(s)
    
    pos_list1= [ stem(d) for d in pos_list] 
    pos_list.extend(pos_list1)
    
    
    neg_list=[]
    for s in negative_words:
        s=s.replace("\n", "")
        neg_list.append(s)
    
    neg_list1= [ stem(d) for d in neg_list] 
    neg_list.extend(neg_list1)
    
    return pos_list,neg_list
        
        
                  

def getDocs(filename="train.dat"):
    # open docs file and read its lines
    with open(filename, "r") as fh:
        train_lines = fh.readlines()
        
    pos_list, neg_list = getPositiveNegativeWords()

    if(filename=="train.dat"):
        train_labels=  [l.split()[0] for l in train_lines]
        train_doc = [re.sub(r'[^\w]', ' ',l[2:].lower()).split() for l in train_lines]
        train_rev = filterLen(train_doc, 4)
        train_documents = stemDoc(train_rev)
        print len(train_documents[0])
        for t in train_documents:
            pos_count=0
            neg_count=0
            new_term="ann-neutral"

            for d in t:
                if(d in pos_list):
                    pos_count+=1
                elif(d in neg_list):
                    neg_count+=1
            if(neg_count>pos_count):
                new_term="ann-neg"
            elif(neg_count<pos_count):
                new_term="ann-pos"
                
                
            new_list = get_k_mers(t)
            t.append(new_term)
 
            t.extend(new_list)
        
        print len(train_documents[0])

    return train_labels, train_documents, pos_list,neg_list 

def splitTrainData(train_docs,train_labels):
    
    train_docs1 = train_docs[0:len(train_docs)/2]
    train_labels1 = train_labels[0:len(train_labels)/2]
    
    test_docs = train_docs[len(train_docs)/2:]
    test_labels = train_labels[len(train_labels)/2:]

    return train_docs1, train_labels1, test_docs, test_labels



def getTestDocs(filename="test.dat"):
    # open docs file and read its lines
    with open(filename, "r") as fh:
        test_lines = fh.readlines()


    test_doc = [re.sub(r'[^\w]', ' ',l[2:].lower()).split() for l in test_lines]
    test_rev = filterLen(test_doc, 4)
    test_documents = stemDoc(test_rev)
    print len(test_documents[0])
    
    for t in test_documents:
        pos_count=0
        neg_count=0
        new_term="ann-neutral"
        for d in t:
            if(d in pos_list):
                pos_count+=1
            elif(d in neg_list):
                neg_count+=1
        if(neg_count>pos_count):
                new_term="ann-neg"
        elif(neg_count<pos_count):
                new_term="ann-pos"
        new_list = get_k_mers(t)
        t.append(new_term)
        t.extend(new_list)
        
    print len(test_documents[0])
        
    return test_documents


# Group words for different K values for K-mer implementation
def grouper(input_list, n = 2):
    for i in xrange(len(input_list) - (n - 1)):
        yield input_list[i:i+n]

def get_k_mers(input_list):
    new_list = []
    #new_list.extend(input_list)
    for first, second in grouper(input_list, 2):
        st = first + " "+second
        new_list.append(st)

    return new_list


def kmer(lists,k=3):
    f = []
    for word in lists:       
        for x in range(len(word)+1-k):  
            kmer = word[x:x+k]
            f.append(kmer)
    return f







In [ ]:
print train_documents[6]

In [ ]:
train_labels, train_documents,pos_list,neg_list  = getDocs()



print train_documents[1]

159


In [18]:

train_documents, train_labels, test_documents, test_labels=splitTrainData(train_documents,train_labels)
train_documents.extend(test_documents)

In [ ]:
test_documents = getTestDocs("test.dat")
train_documents.extend(test_documents)

In [ ]:
print len(test_documents)
print len(train_documents)

In [ ]:
print len(test_documents)

In [32]:


def build_matrix(docs, pos_list,neg_list, weight=2):
    r""" Build sparse matrix from a list of documents, 
    each of which is a list of word/terms in the document.  
    """
        
    nrows = len(docs)
    idx = {}
    tid = 2
    nnz = 0
    idx["ann-neg"]=0
    idx["ann-pos"]=1

    for d in docs:
        nnz += len(set(d))
        for w in d:
            if w not in idx:
                idx[w] = tid
                tid += 1
    ncols = len(idx)
        
    # set up memory
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=np.int)
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in docs:
        cnt = Counter(d)
        #print cnt
        keys = list(k for k,_ in cnt.most_common())
        #print keys
        l = len(keys)
        for j,k in enumerate(keys):
            ind[j+n] = idx[k]
            val[j+n] = cnt[k]
            if(k in pos_list or k in neg_list):
                val[j+n] = val[j+n] * weight
            if(k =='ann-neg' or k =='ann-pos'):
                val[j+n]=val[j+n]*15
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
            
    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat,idx


def build_test_matrix(docs,pos_list,neg_list,idx, weight=3):
    r""" Build sparse matrix from a list of documents, 
    each of which is a list of word/terms in the document.  
    """
    nrows = len(docs)

    
    nnz = 0
    test_docs = []
    for d in docs:
        copy_d=[]
     
        for w in d:
            if w in idx:
                copy_d.append(w)
        
        nnz += len(set(copy_d))
        test_docs.append(copy_d)
        
        
    ncols = len(idx)
        
    # set up memory
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=np.int)
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in test_docs:
        cnt = Counter(d)
        #print cnt
        keys = list(k for k,_ in cnt.most_common())
        #print keys
        l = len(keys)
        for j,k in enumerate(keys):
            ind[j+n] = idx[k]
            val[j+n] = cnt[k]
            if(k in pos_list or k in neg_list):
                val[j+n] = val[j+n] * weight  
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
            
    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat


def csr_idf(mat, copy=False, **kargs):
    r""" Scale a CSR matrix by idf.
    Returns scaling factors as dict. If copy is True,
    returns scaled matrix and scaling factors.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # document frequency
    df = defaultdict(int)
    for i in ind:
        if i>1:
            df[i] += 1
    # inverse document frequency
    for k,v in df.items():
        df[k] = np.log(nrows / float(v))  ## df turns to idf - reusing memory
    # scale by idf
    for i in range(2, nnz):
        val[i] *= df[ind[i]]

    return df if copy is False else mat


def csr_l2normalize(mat, copy=False, **kargs):
    r""" Normalize the rows of a CSR matrix by their L-2 norm. 
    If copy is True, returns a copy of the normalized matrix.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # normalize
    for i in range(nrows):
        rsum = 0.0    
        for j in range(ptr[i], ptr[i+1]):
            rsum += val[j]**2
        if rsum == 0.0:
            continue  # do not normalize empty rows
        rsum = 1.0/np.sqrt(rsum)
        for j in range(ptr[i], ptr[i+1]):
            val[j] *= rsum
            
    if copy is True:
        return mat

def csr_info(mat, name="", non_empy=False):
    r""" Print out info about this CSR matrix. If non_empy, 
    report number of non-empty rows and cols as well
    """
    if non_empy:
        print("%s [nrows %d (%d non-empty), ncols %d (%d non-empty), nnz %d]" % (
                name, mat.shape[0], 
                sum(1 if mat.indptr[i+1] > mat.indptr[i] else 0 
                for i in range(mat.shape[0])), 
                mat.shape[1], len(np.unique(mat.indices)), 
                len(mat.data)))
    else:
        print( "%s [nrows %d, ncols %d, nnz %d]" % (name, 
                mat.shape[0], mat.shape[1], len(mat.data)))
        
        


   



In [34]:
csr_mat,word_dict = build_matrix(train_documents,pos_list,neg_list)

print "done csr"

done csr


In [35]:
print csr_mat[0]

  (0, 0)	15.0
  (0, 2)	2.0
  (0, 3)	1.0
  (0, 4)	1.0
  (0, 5)	1.0
  (0, 6)	1.0
  (0, 7)	2.0
  (0, 8)	2.0
  (0, 9)	2.0
  (0, 10)	2.0
  (0, 11)	1.0
  (0, 12)	1.0
  (0, 13)	1.0
  (0, 14)	2.0
  (0, 15)	2.0
  (0, 16)	1.0
  (0, 17)	4.0
  (0, 18)	1.0
  (0, 19)	1.0
  (0, 20)	1.0
  (0, 21)	4.0
  (0, 22)	1.0
  (0, 23)	4.0
  (0, 24)	1.0
  (0, 25)	2.0
  :	:
  (0, 252)	1.0
  (0, 253)	1.0
  (0, 254)	1.0
  (0, 255)	1.0
  (0, 256)	1.0
  (0, 257)	1.0
  (0, 258)	1.0
  (0, 259)	1.0
  (0, 260)	1.0
  (0, 261)	1.0
  (0, 262)	1.0
  (0, 263)	1.0
  (0, 264)	1.0
  (0, 265)	1.0
  (0, 266)	1.0
  (0, 267)	1.0
  (0, 268)	1.0
  (0, 269)	1.0
  (0, 270)	1.0
  (0, 271)	1.0
  (0, 272)	1.0
  (0, 273)	1.0
  (0, 274)	1.0
  (0, 275)	1.0
  (0, 276)	1.0


In [36]:
mat1 = csr_idf(csr_mat, copy=True)

In [37]:
mat=csr_l2normalize(mat1,copy=True)

In [ ]:
tmat = build_test_matrix(test_documents,word_dict)
csr_l2normalize(tmat)
print tmat.shape

In [38]:
print mat.shape

(25000, 1480047)


In [45]:
def getNeighborsWhole(distances,train_labels,train_no,test_no, k):
    train_no=12500
    test_labels=[]

    for index in range(train_no, train_no+test_no):
        similarity = distances[index, :train_no].toarray().tolist()[0]

        zipped_sim_labels = zip(similarity, train_labels, range(len(train_labels)))

        sorted_zipped_sim_labels = sorted(zipped_sim_labels, key=lambda (val, k, l): val, reverse=True)
        tmp = 0

        for j in range(k):
            if sorted_zipped_sim_labels[j][0] > 0:
                tmp += int(sorted_zipped_sim_labels[j][1])
        if tmp == 0:
            #get nearest one in case of tie
            tmp = np.random.randint(-1,2)
        if tmp > 0:
            test_labels.append('+1')
        else:
            test_labels.append('-1')
   
    return test_labels    

In [43]:
distances = cosine_similarity(mat,dense_output=False)



In [46]:
 

    
def classify(x, train, clstr,k=3):
        r""" Classify vector x using kNN and majority vote rule given training data and associated classes
        """
        # find nearest neighbors for x
        dots = x.dot(train.T)
        sims = list(zip(dots.indices, dots.data))
        if len(sims) == 0:
            # could not find any neighbors
            return '+' if np.random.rand() > 0.5 else '-'
        sims.sort(key=lambda x: x[1], reverse=True)
        tc = Counter(clstr[s[0]] for s in sims[:k]).most_common(2)
        if len(tc) < 2 or tc[0][1] > tc[1][1]:
            # majority vote
            return tc[0][0]
        # tie break
        tc = defaultdict(float)
        for s in sims[:k]:
            tc[clstr[s[0]]] += s[1]
        return sorted(tc.items(), key=lambda x: x[1], reverse=True)[0][0]
        

        
macc = 0.0
d=10



#svd = TruncatedSVD(n_components=5000, n_iter=7, random_state=42)

#scaled_mat = svd.fit_transform(mat)


k_list =[17,33,349,399,449]
#k_list =[449]



for f in k_list:
    

        # split data into training and testing
        # predict the class of each test sample
    print "calculating for K="+str(f)
    clspr = getNeighborsWhole(distances, train_labels,len(train_documents),len(test_documents),f)
    #print clspr

    #clspr = [ classify(test[i,:], train, clstr,f) for i in range(test.shape[0]) ]
        # compute the accuracy of the prediction
    acc = 0.0
    for i in range(len(test_labels)):
        if test_labels[i] == clspr[i]:
            acc += 1
    acc /= len(test_labels)
    print str(f)+"--->"+str(acc)
    macc += acc
        
print macc/(31-9)

calculating for K=17
17--->0.7872
calculating for K=33
33--->0.80472
calculating for K=349


KeyboardInterrupt: 

In [ ]:

    

import os
filename = 'output.dat'
highscore = open(filename,'w')
num_rows, num_cols = tmat.shape
print num_rows

clspr = getNeighborsWhole(mat,tmat,99)

for e in clspr:
    
    highscore.write(e+"\n")

highscore.close()  

In [ ]:

from itertools import product
from string import ascii_lowercase
keywords = map(''.join, product("hello", repeat=3))
print keywords

In [ ]:
mat1 =[[1,2][3,4]]
mat2 =[[8,9],[5,7],[1,2]]
    
    
    
    
distances=mat1.dot(mat2.T)
distances =np.array(distances)
indixes= np.argsort(distances,axis=1)[::-1]

print distances

print indixes

In [ ]:
mat1 =np.array([[1,90],[3,4]])
mat2 =np.array([[8,9],[5,7],[1,2],[6,7],[8,4]])
train_labels=['-1','+1','-1','+1','-1']

clspr = getNeighborsWhole(mat2,mat1,3)

print clspr



In [ ]:
# Group words for different K values for K-mer implementation
def grouper(input_list, n = 2):
    for i in xrange(len(input_list) - (n - 1)):
        yield input_list[i:i+n]

def get_k_mers(input_list):
    new_list = []
    #new_list.extend(input_list)
    for first, second in grouper(input_list, 2):
        st = first + " "+second
        new_list.append(st)

    for first, second, third in grouper(input_list, 3):
        st = first + " "+second + " "+third
        new_list.append(st)

    return new_list

In [ ]:
list=["abcde","rashmi","popopopoop","whst i "]


get_k_mers(list)

In [ ]:
l=[]
l.append(["abcde","rashmi","popopopoop","whst i ","this","ss"])
l.append(["abcde","rashmi","popopopoop","whst i ","this"])
print 'this'  in l[0]

build_matrix(l,3)

